In [1]:
from itertools import islice

from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from tqdm.autonotebook import tqdm

import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from gluonts.dataset.pandas import PandasDataset
import pandas as pd

from lag_llama.gluon.estimator import LagLlamaEstimator

C:\Users\abiza\AppData\Local\Temp\ipykernel_17528\45153676.py:5: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
c:\Users\abiza\AppData\Local\Programs\Python\Python311\Lib\site-packages\gluonts\json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


# Compared to TFT (Temporal Fusion Transformer) and DeepAR

In [ ]:
%pip install yfinance

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

from gluonts.dataset.pandas import PandasDataset
from gluonts.torch import DeepAREstimator, TemporalFusionTransformerEstimator
# from gluonts.trainer import Trainer
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.common import ListDataset
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

from tqdm.autonotebook import tqdm
import yfinance as yf

In [3]:
# Contoh simbol saham, gantikan dengan daftar simbol yang valid
symbols = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'JPM', 'JNJ', 'V', 'PG', 'UNH', 'MA', 'INTC', 'VZ', 'HD', 'T', 'DIS', 'BAC', 'XOM', 'PFE', 'KO', 'CSCO', 'CMCSA', 'PEP', 'BA', 'WFC', 'MCD', 'COST', 'CVX', 'ABT', 'LLY', 'NKE', 'MDT', 'IBM', 'NEE', 'DHR', 'HON', 'ACN', 'TXN', 'GILD', 'LMT', 'BMY', 'SBUX', 'FIS', 'AMGN', 'MO', 'INTU', 'UNP', 'MMM', 'CHTR', 'TMO', 'LOW', 'BKNG', 'UPS', 'ADBE', 'QCOM', 'ORCL', 'MS', 'BLK', 'GS', 'NOW', 'SCHW', 'MDLZ', 'GE', 'C', 'FDX', 'CAT', 'AXP', 'SPGI', 'TGT', 'ISRG', 'DE', 'SYK', 'NSC', 'MU', 'SO', 'GM', 'ADP', 'VRTX', 'ZTS', 'USB', 'CI', 'GD', 'REGN', 'RTX', 'DUK', 'PGR', 'CL', 'CVS', 'D', 'CSX', 'TFC', 'COP', 'SPG', 'EL', 'MMC', 'SHW', 'BIIB']

stock_prices_2 = (
    yf.Tickers(symbols)
    .history(period="max", start="2013-01-01")
    .Close
    .resample('1d')
    .ffill()
)

stock_returns_2 = stock_prices_2.rolling(5).mean().pct_change().dropna()

stock_returns_2

[*********************100%***********************]  96 of 96 completed


Ticker,AAPL,ABT,ACN,ADBE,ADP,AMGN,AMZN,AXP,BA,BAC,...,UNH,UNP,UPS,USB,V,VRTX,VZ,WFC,XOM,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2013-02-06 00:00:00+00:00,0.001650,0.001361,-0.003754,0.000313,0.000396,0.000672,-0.002103,0.002168,0.003780,0.003761,...,0.005733,-0.001079,0.002346,0.001263,0.002854,0.001854,0.000853,-0.000914,-0.000557,0.000129
2013-02-07 00:00:00+00:00,0.007651,0.003074,-0.002321,-0.001409,-0.000495,-0.002503,-0.003616,0.007422,0.006788,0.002214,...,0.004532,-0.002942,0.004557,0.000841,-0.001889,0.000925,-0.000449,-0.002688,-0.002733,0.006383
2013-02-08 00:00:00+00:00,0.010573,0.003182,-0.001341,0.004075,0.000925,0.001092,-0.002321,0.006244,0.004451,0.000850,...,0.006943,-0.001219,0.005974,0.001500,-0.000959,0.002465,-0.000942,-0.001434,-0.001930,0.013069
2013-02-09 00:00:00+00:00,0.015362,0.005345,0.001343,0.005307,0.004555,0.002391,0.001502,0.007847,0.003540,0.004755,...,0.006966,0.001748,0.006087,0.004135,0.001452,0.006016,-0.000764,0.000689,0.000059,0.012837
2013-02-10 00:00:00+00:00,0.008502,0.003330,-0.000137,0.003157,-0.000986,0.000556,-0.003762,0.003714,0.001751,-0.002028,...,-0.000104,0.000015,0.003870,0.003103,-0.002181,0.001746,-0.000944,0.000172,-0.001262,0.012549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-21 00:00:00+00:00,0.003101,0.002336,0.004224,0.000427,0.004653,0.006060,-0.004170,0.004279,0.004498,-0.001246,...,0.001782,0.002943,-0.001608,0.005923,0.000168,-0.006672,0.004053,0.001325,0.004382,0.001788
2024-11-22 00:00:00+00:00,0.004276,0.003187,0.002862,0.003503,0.005019,0.009289,-0.005435,0.010030,0.012654,0.001076,...,-0.000460,0.005800,0.003583,0.010236,0.000181,-0.006454,0.007124,0.004375,0.004130,0.002069
2024-11-23 00:00:00+00:00,0.001618,0.000683,0.003291,0.005025,0.004828,0.011020,-0.004559,0.010846,0.007442,0.001204,...,0.000413,0.006747,-0.000465,0.007354,-0.001444,-0.007074,0.004244,0.005028,0.002454,0.000613


In [5]:
def _get_lag_llama_dataset(dataset):
    dataset = dataset.copy()

    for col in dataset.columns:
        if dataset[col].dtype != "object" and not pd.api.types.is_string_dtype(dataset[col]):
            dataset[col] = dataset[col].astype("float32")

    backtest_dataset = PandasDataset(dict(dataset))
    return backtest_dataset

train_dataset_2 = _get_lag_llama_dataset(stock_returns_2.iloc[:int(0.7*len(stock_returns_2))])
test_dataset_2 = _get_lag_llama_dataset(stock_returns_2.iloc[int(0.7*len(stock_returns_2)):])

prediction_length = 60
num_samples = 1060
device = torch.device("cuda:0")

In [6]:
deepar_estimator = DeepAREstimator(
  prediction_length=prediction_length, 
  context_length=32, 
  lr=0.0001,
  freq="D",
  num_parallel_samples=num_samples,
  batch_size=64,
  nonnegative_pred_samples=True, 
  trainer_kwargs={"max_epochs": 50})

In [ ]:
deepar_predictor = deepar_estimator.train(train_dataset_2)

In [8]:
# Make predictions
deepar_forecast_it, deepar_ts_it = make_evaluation_predictions(
  dataset=test_dataset_2, 
  predictor=deepar_predictor) 

deepar_forecasts = list(deepar_forecast_it) 
deepar_tss = list(deepar_ts_it) 

evaluator = Evaluator()
# Get evaluation metrics
deepar_agg_metrics, deepar_ts_metrics = evaluator(iter(deepar_tss), iter(deepar_forecasts)) 

print('MSE:',deepar_agg_metrics['MSE'])
print('MAPE:',deepar_agg_metrics['MAPE'])
print('CRPS:',deepar_agg_metrics['mean_wQuantileLoss'])

c:\Users\abiza\AppData\Local\Programs\Python\Python311\Lib\site-packages\gluonts\dataset\pandas.py:174: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df = df.to_period(freq=self.freq)
c:\Users\abiza\AppData\Local\Programs\Python\Python311\Lib\site-packages\gluonts\dataset\pandas.py:174: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df = df.to_period(freq=self.freq)
c:\Users\abiza\AppData\Local\Programs\Python\Python311\Lib\site-packages\gluonts\dataset\pandas.py:174: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df = df.to_period(freq=self.freq)
c:\Users\abiza\AppData\Local\Programs\Python\Python311\Lib\site-packages\gluonts\dataset\pandas.py:174: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df = df.to_period(freq=self.freq)
c:\Users\abiza\AppData\Local\Programs\Python\Python311\Lib\site-packages

MSE: 4.475485511648003e-05
MAPE: 1.4722681492545775
CRPS: 0.8707829794657858


In [9]:
tft_estimator = TemporalFusionTransformerEstimator(
  prediction_length=prediction_length, 
  context_length=32, 
  lr=0.0001,
  freq="D", 
  batch_size=64,
  trainer_kwargs={"max_epochs": 50}) 

In [ ]:
tft_predictor = tft_estimator.train(train_dataset_2)

In [11]:
tft_forecast_it, tft_ts_it = make_evaluation_predictions(
  dataset=test_dataset_2, 
  predictor=tft_predictor) 

tft_forecasts = list(tft_forecast_it) 
tft_tss = list(tft_ts_it) 

tft_agg_metrics, tft_ts_metrics = evaluator(iter(tft_tss), iter(tft_forecasts)) 

print('MSE:',tft_agg_metrics['MSE'])
print('MAPE:',tft_agg_metrics['MAPE'])
print('CRPS:',tft_agg_metrics['mean_wQuantileLoss'])

c:\Users\abiza\AppData\Local\Programs\Python\Python311\Lib\site-packages\gluonts\dataset\pandas.py:174: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df = df.to_period(freq=self.freq)
c:\Users\abiza\AppData\Local\Programs\Python\Python311\Lib\site-packages\gluonts\dataset\pandas.py:174: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df = df.to_period(freq=self.freq)
c:\Users\abiza\AppData\Local\Programs\Python\Python311\Lib\site-packages\gluonts\dataset\pandas.py:174: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df = df.to_period(freq=self.freq)
c:\Users\abiza\AppData\Local\Programs\Python\Python311\Lib\site-packages\gluonts\dataset\pandas.py:174: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df = df.to_period(freq=self.freq)
c:\Users\abiza\AppData\Local\Programs\Python\Python311\Lib\site-packages

MSE: 4.186407352588301e-05
MAPE: 2.362294517177538
CRPS: 0.8071181979885231


In [ ]:
print(tft_forecasts)

In [ ]:
hasil_msee = {
    'ARIMA' : 0.0000496185,
    'Lag-Llama' : 0.0000263849,
    'DeepAR' : 0.0000447193,
    'TFT' : 0.0000424322
}

colors = ['blue', 'green', 'red', 'purple']

plt.figure(figsize=(8, 5))
bars = plt.bar(list(hasil_msee.keys()), list(hasil_msee.values()), color=colors)
plt.title('Perbandingan Mean Squared Error (MSE)')
plt.xlabel('Model')
plt.ylabel('MSE')
plt.yscale('log')  # Log scale for better visualization of small values

# Adding MSE values to the top of each bar
for bar, value in zip(bars, hasil_msee.values()):
    plt.text(bar.get_x() + bar.get_width() / 2, value, f'{value:.8f}', va='bottom', ha='center')

plt.show()

In [ ]:
hasil_mape = {
    'ARIMA' : 5.7117,
    'Lag-Llama' : 2.2518,
    'DeepAR' : 1.62233,
    'TFT' : 2.55507
}

colors = ['blue', 'green', 'red', 'purple']

plt.figure(figsize=(8, 5))
bars = plt.bar(list(hasil_mape.keys()), list(hasil_mape.values()), color=colors)
plt.title('Perbandingan Mean Absolute Percentage Error (MAPE)')
plt.xlabel('Model')
plt.ylabel('MAPE')
plt.yscale('log')  # Log scale for better visualization of small values

# Adding MSE values to the top of each bar
for bar, value in zip(bars, hasil_mape.values()):
    plt.text(bar.get_x() + bar.get_width() / 2, value, f'{value:.4f}', va='bottom', ha='center')

plt.show()